# Travel planner

## Функции для взаимодействия с gmaps:

In [11]:
from itertools import product
import ipywidgets as widgets
import geopy
import gmaps, googlemaps
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY
import numpy as np
import gmaps
%matplotlib inline
gmaps.configure(api_key=Your_Key)

In [12]:
Your_key = input("Enter your API key, please ")
api_key =Your_Key
import requests, json 
url = 'http://photon.komoot.de/api/?q=' 
def coord(places): # вход: лист названий мест, выход; лист координат этих местов
    numb = len(places)
    coord = [(0,0)]*numb
    for i in range (numb): 
        places[i]= places[i].replace(' ','+')
        resp = requests.get(url=url+places[i]) 
        data = json.loads(resp.text) 
        coord[i]=(data['features'][0]['geometry']['coordinates'][1],data['features'][0]['geometry']['coordinates'][0])
    return coord
def dist(origin, destination): #вход: название двух точек, выход: расстояние между ними в метрах
    origin = origin.replace(' ','+')
    destination = destination.replace(' ','+')
    gmaps = googlemaps.Client(key=Your_key) # Requires cities name 
    distance = gmaps.distance_matrix(origin,destination, mode='walking')['rows'][0]['elements'][0] 
    return distance['distance']['value']
def matr_dist(places): #en: лист названия точек, ex: матрица расстояний между ними
    numb= len(places)
    c = np.zeros((numb,numb))
    for i in range (numb):
        for j in range(i,numb):
            c[i][j] = dist(places[i], places[j])
        if i>0:
            for j in range (i):
                c[i][j] = c[j][i]
    return c

In [13]:

#Графические функции
def draw_problem(places):#вход: название точек маршрута, выход: визуал. точек маршрута на карте,  
    import gmaps
    gmaps.configure(api_key=Your_key)
    fig = gmaps.figure()
    drawing = gmaps.symbol_layer(coord([places[0]]), fill_color = 'green', hover_text = 'Star and End point', fill_opacity =1.0,stroke_color = 'green')
    fig.add_layer(drawing) 
    for i in range(1, len(places)):
        a= coord([places[i]])[0]
        drawing = gmaps.drawing_layer(features=[gmaps.Marker(a)])
        fig.add_layer(drawing) 
    return fig   
    
def draw_solution(route): #вход: название упорядоченных точек машрута, выход: визуал маршрута на gmaps
               #из-за ограничений gmaps api маршрут будет проведен только между первыми 8ьмю точками
    import gmaps
    gmaps.configure(api_key=Your_key)
    fig = gmaps.figure()
    for i in range(len(route) - 1):
        a= coord([route[i]])[0]
        if (i<7):
            directions = gmaps.directions_layer(coord([route[i]])[0], coord([route[i+1]])[0], show_markers=False, travel_mode='WALKING')
            #directions = gmaps.directions_layer(gmaps.locations.locations_to_list(route[i])[0], gmaps.locations.locations_to_list(route[i+1])[0], show_markers=False, travel_mode='WALKING')  
            fig.add_layer(directions)
        drawing = gmaps.drawing_layer(features=[gmaps.Marker(a, label=str(i))])
        fig.add_layer(drawing)
    return fig
def draw_solution_coords(route): #вход: название упорядоченных точек машрута, выход: визуал маршрута на gmaps
               #из-за ограничений gmaps api маршрут будет проведен только между первыми 8ьмю точками
    import gmaps
    gmaps.configure(api_key=Your_key)
    fig = gmaps.figure()

    for i in range( len(route) -1):
        if (i<9):
            #directions = gmaps.directions_layer(route[i], route[i+1], show_markers=False, travel_mode='WALKING')
            directions = gmaps.directions_layer(route[i], route[i+1], show_markers=False, travel_mode='WALKING')
            fig.add_layer(directions)
        drawing = gmaps.drawing_layer(features=[gmaps.Marker(route[i], label=str(i))])
        fig.add_layer(drawing)
    return fig

### Взаимодейтсвие с пользователем

In [14]:
api_key = Your_key
class ReverseGeocoder(object): #заносит в массив places геолокации выбранные пользователем на карте
    def __init__(self, fig):
        self._figure = fig #gmaps.figure(center=(55.0, 5.0), zoom_level=8)
        self._drawing = gmaps.drawing_layer()
        self._drawing.on_new_feature(self._new_feature_callback)
        self._figure.add_layer(self._drawing)
        self._address_box = widgets.Text(
            description='Address: ',
            disabled=True,
            layout={'width': '95%', 'margin': '10px 0 0 0'}
        )
        #self._button = widgets.ToggleButtons(ptions=['Finish'], description='Are you done?:', disabled=False, button_style='')
        
        self._geocoder = geopy.geocoders.GoogleV3(api_key=api_key)
        self._container = widgets.VBox([self._figure, self._address_box])
    def _show_address(self, location):
        location_details = self._get_location_details(location)
        if location_details is None:
            self._address_box.value = 'No address found'
        else:
            #places.append(self._address_box.value)
            self._address_box.value =location_details.address
            
            places.append(self._address_box.value)
        print ("Адрес:", self._address_box.value)

    def _get_location_details(self, location):
        return self._geocoder.reverse(location, exactly_one=True)
    def _clear_address_box(self):
        self._address_box.value = ''
    def _new_feature_callback(self, feature):
        try:
            location = feature.location
        except AttributeError:
            return # Not a marker
        
# Clear address box to signify to the user that something is happening
        self._clear_address_box()
# Remove all markers other than the one that has just been added.
        self._drawing.features = [feature]
# Compute the address and display it
        self._show_address(location)
        
    def render(self):
        return self._container


import urllib.request, json
def get_places_from_user(): #получить лист мест, который хотит посетить пользователь
    api_key = Your_key

    #Google MapsDdirections API endpoint
    endpoint = 'https://maps.googleapis.com/maps/api/directions/json?'

    #Asks the user to input Where they are and where they want to go.
    a = input('Where are you?: ').replace(' ','+')
    places = []
    coords = []
    #how = input('What mode of transport do you use? (legs/driving): ').replace(' ','+')
    print('When you are done, enter "End"')
    while ((a == 'End') == False):
        places.append(a)
        a = input('What else do you want to visit?: ').replace(' ','+')
    fig = draw_problem(places)
    c = input('Would you like to use map? (Yes/No): ')
    if (c=='Yes'):
        print('If you want to add something, choose on map')
    return places, fig


## Методы решения TSP: 

### Branch and bound method

In [15]:

def tsp_bb(places): # en: лиcт названий мест, ex: route: list  с упорядоченными в опт порядке и итоговой длины пути(в км до 2го знака после запятой),
                   #печать маршрута и минимальной дистанции
                   #с помощью миним obj function методом branch and bound
    c = matr_dist(places)
    # number of nodes and list of vertices
    n, V = len(places), set(range(len(places)))

    model = Model()

    # binary variables indicating if arc (i,j) is used on the route or not
    x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

    # continuous variable to prevent subtours: each city will have a
    # different sequential id in the planned route except the first one
    y = [model.add_var() for i in V]

    model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V))

    for i in V:
        model += xsum(x[i][j] for j in V - {i}) == 1

    for i in V:
        model += xsum(x[j][i] for j in V - {i}) == 1

    for (i, j) in product(V - {0}, V - {0}):
        if i != j:
            model += y[i] - (n+1)*x[i][j] >= y[j]-n

    model.optimize(max_seconds=100)

    out.write('Found route with total distance %g km : %s '% (round(model.objective_value/1000,2), places[0]))
    aft= 0
    i = 1
    route = ['0'] * (len(places)+1)
    route[0] = places[0]
    while True:
        aft = [i for i in V if x[aft][i].x >= 0.99][0]
        out.write(' -> %s' % places[aft])
        route[i] = places[aft]
        i = i+ 1
        if aft == 0:
            break
    out.write('\n')
    return route, round(model.objective_value/1000,2)

### Genetic algorithm

In [16]:
import random
import copy

class State:

    # Create a new state
    def __init__(self, route:[], distance:int=0):
        self.route = route
        self.distance = distance

    # Compare states
    def __eq__(self, other):
        for i in range(len(self.route)):
            if(self.route[i] != other.route[i]):
                return False
        return True

    # Sort states
    def __lt__(self, other):
         return self.distance < other.distance

    # Print a state
    def __repr__(self):
        return ('({0},{1})\n'.format(self.route, self.distance))

    # Create a shallow copy
    def copy(self):
        return State(self.route, self.distance)

    # Create a deep copy
    def deepcopy(self):
        return State(copy.deepcopy(self.route), copy.deepcopy(self.distance))

    # Update distance
    def update_distance(self, MatrDist, place0):
        
        # Reset distance
        self.distance = 0

        # top track of departing place
        from_index = place0

        # Loop all places in the current route
        for i in range(len(self.route)):
            self.distance += MatrDist[from_index][self.route[i]]
            from_index = self.route[i]

        # Add the distance back to place0
        self.distance += MatrDist[from_index][place0]

# This class represent a place (used when we need to delete places)
class place:

    def __init__(self, index:int, distance:int):
        self.index = index
        self.distance = distance

    def __lt__(self, other):
         return self.distance < other.distance


# Create a population
def create_population(MatrDist:[], place0:int, placeInd:[], size:int):

    gene = placeInd.copy()
    gene.pop(place0)
    population = []
    for i in range(size):
        random.shuffle(gene)
        state = State(gene[:])
        state.update_distance(MatrDist, place0)
        population.append(state)
    return population

def crossover(MatrDist:[], place0:int, parents:[]):

    parent1 = parents[0].deepcopy()
    parent2 = parents[1].deepcopy()

    part_1 = []
    part_2 = []
    a = int(random.random() * len(parent1.route))
    b = int(random.random() * len(parent2.route))
    start_gene = min(a, b)
    end_gene = max(a, b)
    for i in range(start_gene, end_gene):
        part_1.append(parent1.route[i])
    
    part_2 = [int(x) for x in parent2.route if x not in part_1]
    state = State(part_1 + part_2)
    state.update_distance(MatrDist, place0)

    # Return a child
    return state

# Mutate a solution
def mutate(MatrDist:[], place0:int, state:State, mutation_rate:float=0.01):
    
    mutated_state = state.deepcopy()

    for i in range(len(mutated_state.route)):
        if(random.random() < mutation_rate):

            # Swap two places
            j = int(random.random() * len(state.route))
            place_1 = mutated_state.route[i]
            place_2 = mutated_state.route[j]
            mutated_state.route[i] = place_2
            mutated_state.route[j] = place_1
    mutated_state.update_distance(MatrDist, place0)
    return mutated_state

# A genetic algorithm
def genetic_algorithm(MatrDist:[], place0:int, population:[], top:int, mutation_rate:float, generations:int):
    
    for i in range(generations):
        population.sort()
        parents = []
        for j in range(1, len(population)):
            parents.append((population[j-1], population[j]))
        children = []
        for partners in parents:
            children.append(crossover(MatrDist, place0, partners))

        for j in range(len(children)):
            children[j] = mutate(MatrDist, place0, children[j], mutation_rate)
 

        population = population[:top]

        population.extend(children)

    population.sort()

    return population[0]

def tsp_ga(places):

    placeInd = [0]*len(places)
    for i in range (len(places)):
        placeInd[i] = i

    place0 = 0 
    MatrDist = matr_dist(places)
    population = create_population(MatrDist, place0, placeInd, 100)
    state = genetic_algorithm(MatrDist, place0, population, 30, 0.02, 100)
    out.write('Found route with total distance %g km : %s '% (round(state.distance/1000,2), places[place0]))
    
    route = ['0'] * (len(state.route)+2)    
    route[0] = places[place0]
    for i in range(0, len(state.route)):
        route[i+1] = places[state.route[i]]
        
        out.write(' -> %s' % places[state.route[i]])
    out.write(' -> %s' % places[place0])
    route[len(state.route)+1] = places[place0]
    return route, round(state.distance/1000,2)

### Simulated annealing

In [14]:

from satsp import solver
def tsp_sa(places):
    solver.Solve(city_list = None, dist_matrix = matr_dist(places), start_temp = None, stop_temp = None, alpha = None, epochs = None, epoch_length = None,  epoch_length_factor = 1.00, stopping_count = 100, screen_output = True)
    return round(solver.GetBestDist()/1000)


# Plan your journey using above methods:

In [10]:

places, fig = get_places_from_user()

ReverseGeocoder(fig).render()

Where are you?: Novaya Basmannaya
When you are done, enter "End"
What else do you want to visit?: Moscow Kremlin
What else do you want to visit?: Moscow Park Kultury
What else do you want to visit?: Dolgoprudny MIPT
What else do you want to visit?: End
Would you like to use map? (Yes/No): Yes
If you want to add something, choose on map


Адрес: Myasnitskiye Vorota Ploshchad', 1, стр. 3, Moskva, Russia, 101000
Адрес: Sretenskiy bul'var, Moskva, Russia, 101000
Адрес: Novaya Basmannaya Ulitsa, 11, Moskva, Russia, 107078


In [15]:
route, val = tsp_bb(places)

gurobi version 9.0 found
Found route with total distance 59.9 km : Novaya+Basmannaya  -> Myasnitskiye Vorota Ploshchad', 1, стр. 3, Moskva, Russia, 101000 -> Sretenskiy bul'var, Moskva, Russia, 101000 -> Moscow+Park+Kultury -> Moscow+Kremlin -> Dolgoprudny+MIPT -> Novaya Basmannaya Ulitsa, 11, Moskva, Russia, 107078 -> Novaya+Basmannaya


In [27]:
route, val = tsp_ga(places)

Found route with total distance 59.9 km : Novaya+Basmannaya  -> Myasnitskiye Vorota Ploshchad', 1, стр. 3, Moskva, Russia, 101000 -> Sretenskiy bul'var, Moskva, Russia, 101000 -> Moscow+Park+Kultury -> Moscow+Kremlin -> Dolgoprudny+MIPT -> Novaya Basmannaya Ulitsa, 11, Moskva, Russia, 107078 -> Novaya+Basmannaya

In [29]:
print("Destination:", tsp_sa(places))

Simulated Annealing starts with parameters:
Initial Temperature:  29282.14425412835
Cooling Rate:  0.99
Initial Epoch Length:  21
Stopping Criterion: No Improvement for 100 epochs

Epoch  1
Temperature:  29282.14425412835
Epoch Length:  21
Current Dist:  68992.0
Best Dist:  62033.0
Epoch  2
Temperature:  28989.322811587066
Epoch Length:  21
Current Dist:  61964.0
Best Dist:  60190.0
Epoch  3
Temperature:  28699.429583471196
Epoch Length:  21
Current Dist:  61938.0
Best Dist:  60190.0
Epoch  4
Temperature:  28412.435287636483
Epoch Length:  21
Current Dist:  67217.0
Best Dist:  60190.0
Epoch  5
Temperature:  28128.310934760117
Epoch Length:  21
Current Dist:  70189.0
Best Dist:  60190.0
Epoch  6
Temperature:  27847.027825412515
Epoch Length:  21
Current Dist:  61755.0
Best Dist:  60190.0
Epoch  7
Temperature:  27568.55754715839
Epoch Length:  21
Current Dist:  61673.0
Best Dist:  60190.0
Epoch  8
Temperature:  27292.871971686807
Epoch Length:  21
Current Dist:  63406.0
Best Dist:  60190

## Попробуем еще раз

In [34]:
places, fig = get_places_from_user()
draw_problem(places)

Where are you?: Moscow Novaya Basmannaya
When you are done, enter "End"
What else do you want to visit?: Moscow Kremlin
What else do you want to visit?: Moscow Chistye Prudy
What else do you want to visit?: Moscow Tverskaya
What else do you want to visit?: Moscow Park Kultury
What else do you want to visit?: Moscow Bolshoi theatre
What else do you want to visit?: End
Would you like to use map? (Yes/No): No


Figure(layout=FigureLayout(height='420px'))

In [44]:
print (places)
route, val = tsp_bb(places)
draw_solution(route)

['Moscow+Novaya+Basmannaya', 'Moscow+Kremlin', 'Moscow+Chistye+Prudy', 'Moscow+Tverskaya', 'Moscow+Park+Kultury', 'Moscow+Bolshoi+theatre']
Found route with total distance 15.76 km : Moscow+Novaya+Basmannaya  -> Moscow+Bolshoi+theatre -> Moscow+Kremlin -> Moscow+Park+Kultury -> Moscow+Tverskaya -> Moscow+Chistye+Prudy -> Moscow+Novaya+Basmannaya


Figure(layout=FigureLayout(height='420px'))

# Cоздаем датасет из ближайших к пользователю достопримечательностей

In [5]:
import googlemaps
import googleplaces
import json
import pprint
import xlsxwriter
import time
api_key = Your_key
# Define the API Key.
API_KEY =Your_key

# Define the Client
gmaps = googlemaps.Client(key = API_KEY)

In [6]:

import json
import urllib.request as url_req
import time
import pandas as pd
import csv
API_KEY= Your_key
API_NEARBY_SEARCH_URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
PLACES_TYPES = [  ('tourist_attraction',1)] 

import gmaps
import gmaps.datasets
gmaps.configure(api_key=Your_key)
fig = gmaps.figure()
import googlemaps
import googleplaces
import json
import pprint
import xlsxwriter
import time
def find_opening_hours(place_id) :
    gmaps = googlemaps.Client(key = API_KEY)

    i = 0
    opening_hours = ['0']* (len(place_id))
    opening_hours[0] = {'open': {'day': 0, 'time': '0000'}}
    for i in range(1, len(place_id)):
        my_place_id = place_id[i]
        my_fields = ['name','opening_hours']
        places_details  = gmaps.place(place_id= my_place_id , fields= my_fields)
        opening_hours[i] = places_details['result'].get('opening_hours', {"periods":"[{'open': {'day': 0, 'time': '0000'}}]"})["periods"]
    return opening_hours
def request_api(url):
    response = url_req.urlopen(url)
    json_raw = response.read()
    json_data = json.loads(json_raw)
    return json_data

def get_places(types, pages, Center, RADIUS =20000):
    location = str(Center[0]) + "," + str(Center[1])
    mounted_url = ('%s'
        '?location=%s' 
        '&radius=%s'
        '&type=%s'
        '&key=%s') % (API_NEARBY_SEARCH_URL, location, RADIUS, types, API_KEY)
    
    results = []
    next_page_token = 5

    if pages == None: pages = 40

    for num_page in range(pages):
        if num_page < 40:
            api_response = request_api(mounted_url)
            results = results + api_response['results']
        else:
            page_url = ('%s'
                '?key=%s'
                '&pagetoken=%s') % (API_NEARBY_SEARCH_URL, API_KEY, next_page_token)
            api_response = request_api(str(page_url))
            results += api_response['results']

        if 'next_page_token' in api_response:
            next_page_token = api_response['next_page_token']
        else: break
        
        time.sleep(1)
    return results

def parse_place_to_list(place, type_name):
    return [
        
        place['name'],
        place['place_id'],
        place['geometry']['location']['lat'],
        place['geometry']['location']['lng'],
        #place['rating'],
        place['types'],
        place['rating'],
        type_name
    ]

def mount_dataset(Center, radius = 20000):
    data = []

    for place_type in PLACES_TYPES:
        type_name = place_type[0]
        type_pages = place_type[1]
        result = get_places(type_name, type_pages, Center, radius)
        result_parsed = list(map(lambda x: parse_place_to_list(x, type_name), result))
        data += result_parsed

    dataframe = pd.DataFrame(data, columns=['place_name', 'place_id', 'lat', 'lng', 'types', 'rating', 'type'])
    #print(dataframe)
    lat = dataframe['lat'].to_list()
    lng = dataframe['lng'].to_list()
    place_name = dataframe['place_name'].to_list()
    rating = dataframe['rating'].to_list()
    place_id = dataframe['place_id'].to_list()
    places = [(str(lat[0])+','+str(lng[0]))]
    for i in range (1,len(lat)):
        places.append((str(lat[i])+','+str(lng[i])))
    #dataframe.to_csv('places.csv')
    return places, rating, place_name, place_id, lat, lng
def get_places_for_user (home, radius = 20000):
    Center = coord(home)[0]
    places0, rating0, place_name0, place_id0, lat0, lng0 =mount_dataset(Center, radius)
    places = ['0']*(len(places0)+1)
    places[0] = str(Center[0])+','+str(Center[1])
    places[1:] = places0
    Coord = [()]*(len(places0)+1)
    Coord[0] = (Center[0], Center[1])
    for i in range(len(places0)):
        Coord[i+1] =(lat0[i],lng0[i]) 
        #print(len(Coord[i+1]), '\n')
    place_id = ['0']*(len(places0)+1)
    place_id[0] = 0
    place_id[1:] = place_id0
    place_name = ['0']*(len(places0)+1)
    place_name[0] = home[0]
    place_name[1:] = place_name0
    rating = ['0']*(len(rating0)+1)
    rating[0] = 0
    rating[1:] = rating0
    #place_name.insert(0, home)
    #rating.insert = (0,0)
    opening_hours = find_opening_hours(place_id)
    return  places, rating, place_name, opening_hours, place_id, Coord


# Создадим модель для оптимизации 
Для этого построим функцию потерь и добавим ограничения

In [7]:
def on_time (time, day, j): #функция определяющая пришли ли мы в объект j, когда он открыт

    day1 = 0
    i=0
    penalty = 0
    print ("j =", j, "i=", i, "\n")
    #print(opening_hours[1][0]['open']['time'] == '0000' and opening_hours[j][0]['open']['day']=='0') 
    if (opening_hours[1] != [{'open': {'day': '0', 'time': '0000'}}]):
        while day1 != day:
            i = i+1
            if (i > day):
                penalty = 1
                break
            else:
                day1 = int(opening_hours[j][i]['open']['day'])
        if (penalty == 0):
            if (int(opening_hours[j][i]['open']['time'])<=time) and (time <= int(opening_hours[j][i]['close']['time']) ):
                penalty == 0
            else:
                penalty =1
    return penalty
#on_time(1400,1, 2)


In [8]:
#branch and bound methods
#branch and bound methods
#rating = [4, 4,4,3,5,6,7,8,9,10,11,12,13,14,15,16]
places=[]
import time
def tsp_model (places, Coord, rating= [0 for i in range(len(places))], place_name = places, opening_hours =[{'open': {'day': 0, 'time': '0000'}}]*len(places), time= float("inf"), speed=5, day =0, now_time = '9000'): # en: лиcт названий мест, ex: route: list  с упорядоченными в опт порядке и итоговой длины пути(в км до 2го знака после запятой),
                   #печать маршрута и минимальной дистанции
                   #с помощью миним obj function методом branch and bound
    c = matr_dist(places)
    S = 0
    k=0
    # number of nodes and list of vertices
    n, V = len(places), set(range(len(places)))
    for i in V:
        for j in V:
            S = S+c[i][j]
            k = k+1
    mean = S/k
    # number of nodes and list of vertices
    n, V = len(places), set(range(len(places)))

    model = Model()

    # binary variables indicating if arc (i,j) is used on the route or not
    x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

    # continuous variable to prevent subtours: each city will have a
    # different sequential id in the planned route except the first one
    y = [model.add_var() for i in V]
    # objective function: minimize the distance
    model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V) - 2*mean* xsum(( rating[i]-3)*x[i][j] for j in V for i in V ) ) #- 100 * rating[3]*xsum(x[3][j] for j in V))
    model+=xsum((((c[i][j]+speed*500)/(speed*1000)))*x[i][j]  for i in V for j in V) <= time
    # constraint : leave each city only once
    for i in V:
        model += xsum(x[i][j] for j in V - {i}) <= 1
    model += xsum(x[0][j] for j in V) == 1
    t = int(now_time)
    #for j in V: 
        #model += on_time(t,day,j) ==0
           
            
    #model + = xsum(x[i][j] * on_time(get_time(now_time, speed, x, len(places))[j], day, j, x) for j in V - {0} for i in V))

    for i in V:
        model += xsum(x[j][i] for j in V - {i}) <= 1
    #for i in V:
    model += xsum(x[i][i] for i in V - {0}) ==0
    for i in  V:
        model += xsum(x[i][j] - x[j][i] for j in V) ==0
    for (i, j) in product(V - {0}, V - {0}):
        if i != j:
            model += y[i] - (n+1)*x[i][j] >= y[j]-n

    # optimizing
    model.optimize(max_seconds=100)
    
    aft= 0
    i = 1
    route = [Coord[0]]
    #print ('x: \n',x)
    d = [[x[i][j].x for i in V] for j in V]
    rat = 0
    routelength = 0
    out.write('Found route with total distance %g km : %s '% (model.objective_value, place_name[0]))
    total = 0
    for i in V:
        r= 0
        for j in V:
            r = r+d[i][j]
            routelength =routelength + d[i][j]*c[i][j]
        total = total+r
        rat = rat+r*(rating[i]-3)
    j = 0
    while (j < len(places)):
        i = 0
        while (d[j][i]==0):
            i = i+1
        j = i

        out.write ("--> %s" %(place_name[j]))
        route.append( Coord[j])
        if (j == 0):
            break
           
    out.write('\n')
    print ('\n Суммарный рейтинг мест:', rat)
    print ('Длина маршрута:', routelength/1000, 'км' )
    print('Кол-во посещенных мест:', total)
    print('Время на маршрут:', routelength/(1000*speed) + 0.5*total, ' часов')
    return route, routelength/1000, total
#places, rating, place_name, opening_hours, place_id, Coord = get_places_for_user(['Новая Басманная'])
#print (places)
#route, length, total= tsp_model(places[:5],Coord, rating, place_name, opening_hours, 10, 5, 0, '00000')#, 5, rating[:10])
#draw_solution_coords(route)

# Don't know where exactly do you want to go? Plan here!

In [10]:
import time
loc = input("Where are you staying? ")
places, rating, place_name, opening_hours, place_id, Coord = get_places_for_user([loc])
time = input("When do you want to start your tour? (e.g. '1430', if you mean 14:30) ")
day = input ("On which day you want to do the tour? (0 for Sunday, 1 for Monday, e.t.c.)" )
time_limit = input("How long your tour should be (in hours)? ")
speed = input("What is your average speed? (in km/hours, usually 5) ")
route, length, total= tsp_model(places, Coord, rating, place_name, opening_hours, int(time_limit), int( speed))
draw_solution_coords(route)


Where are you staying? Novaya Basmannaya
When do you want to start your tour? (e.g. '1430', if you mean 14:30) 1430
On which day you want to do the tour? (0 for Sunday, 1 for Monday, e.t.c.)0
How long your tour should be (in hours)? 10
What is your average speed? (in km/hours, usually 5) 5
gurobi version 9.0 found
Found route with total distance -193082 km : Novaya+Basmannaya --> Red Square--> Lenin's Mausoleum--> Cathedral of the Archangel--> Annunciation Cathedral--> Cathedral of Christ the Saviour--> Book Museum--> Новоарбатский мост--> Rimsko-Katolicheskiy Kafedral'nyy Sobor Neporochnogo Zachatiya Presvyatoy Devy Marii--> Московский академический Музыкальный театр К. С. Станиславского и В. И. Немировича-Данченко--> Sanduny--> Sretensky Monastery--> Novaya+Basmannaya

 Суммарный рейтинг мест: 15.600000000000001
Длина маршрута: 19.576 км
Кол-во посещенных мест: 12.0
Время на маршрут: 9.9152  часов


Figure(layout=FigureLayout(height='420px'))

# Cравнение методов
Сравним работу ga, sa, bb на большом датасете кафе старбакс в UK

In [21]:
#testing algorithm
import googlemaps
import pandas as pd
import gmaps.datasets
gmaps.configure(api_key=Your_key)
df = gmaps.datasets.load_dataset_as_df('starbucks_kfc_uk')
print (df)
starbucks_df = df[df['chain_name'] == 'starbucks']
starbucks_df = starbucks_df[['latitude', 'longitude']]
gmaps=googlemaps.Client(key=Your_key)
lat = starbucks_df['latitude'].to_list()
lng = starbucks_df['longitude'].to_list()

places = [(str(lat[0])+','+str(lng[0]))]
for i in range (1,len(lat)):
    #geom.append((lat[i], lng[i]))
    places.append((str(lat[i])+','+str(lng[i])))

a22,bb=tsp_bb(places[:30])
a11, ga = tsp_ga(places[:30])
sa = tsp_sa(places[:30])
a1,bb5=tsp_bb(places[30:35])
a2,ga5 = tsp_ga(places[30:35])
sa5 = tsp_sa(places[30:35])
a4, bb10=tsp_bb(places[50:65])
a5,ga10 = tsp_ga(places[50:65])
sa10 = tsp_sa(places[50:65])
print('30 items, sa:', sa, '. ga:', ga , ', bb:', bb)
print('15 items, sa:', sa10, '. ga:', ga10 , ', bb:', bb10)
print('5 items, sa:', sa5, '. ga:', ga5 , ', bb:', bb5)

       latitude  longitude chain_name
0     57.143224  -2.111544  starbucks
1     57.143568  -2.096921  starbucks
2     57.132247  -2.123264  starbucks
3     57.147790  -2.098023  starbucks
4     50.842126  -0.251903  starbucks
...         ...        ...        ...
1217  50.829303  -0.375141        kfc
1218  51.631510  -0.753085        kfc
1219  51.618643  -0.719710        kfc
1220  52.369341  -2.263841        kfc
1221  52.385240  -2.240751        kfc

[1222 rows x 3 columns]
Found route with total distance 3139.1 km : 57.143224,-2.111544  -> 57.14779,-2.098023 -> 57.143568,-2.096921 -> 54.585029,-5.941012 -> 54.338576,-6.181319 -> 54.580289,-5.965722 -> 54.587272,-5.932406 -> 54.598614,-5.928349 -> 54.654844,-5.697606 -> 54.60062,-5.934267 -> 54.661554,-6.216257 -> 54.721915,-6.221398 -> 54.449405,-6.38991 -> 54.599634,-5.927585 -> 51.380831,-2.363118 -> 52.13611,-0.478392 -> 52.022212,1.14223 -> 51.590785,0.479527 -> 51.569273,0.46075 -> 51.143558,0.876705 -> 50.842126,-0.251903 -> 5